In [1]:
from pathlib import Path

import polars as pl
import srsly

In [2]:
raw_tok_path = Path("/home/pl487/rdd/outputs/tok_train/bpe_2024-09-04T12-59-54")

In [3]:
vocab_size = 32_000
tok_type = "bpe"
tok_path = Path("outputs/tokenizers") / f"{tok_type}{vocab_size}"
with (tok_path / "raw_tok_path.txt").open("r") as fl:
    p = fl.readline()

In [4]:
df = pl.DataFrame(srsly.read_jsonl(Path(raw_tok_path) / "implemented_merges.jsonl"))

In [5]:
df = df.with_columns(prefix=pl.col("pair").list.get(0))

In [6]:
df = df.filter(pl.col("new_token_id") < vocab_size)

In [7]:
df = df.group_by("prefix").agg(pl.col("new_token_id")).sort("prefix")

In [8]:
df.write_ndjson(tok_path / f"prefix_map_{tok_type}{vocab_size}.jsonl")

In [9]:
(
    df.with_columns(pl.col("new_token_id").list.min())
    .with_columns(check=pl.col("prefix") >= pl.col("new_token_id"))["check"]
    .sum()
)

0